# Detección de Objetos
- COCO Dataset
- Faster R-CNN model

<b>Fuentes:</b> 
1. [Pytorch tutorial](https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html)
2. [PyTorch object detection with pre-trained networks](https://pyimagesearch.com/2021/08/02/pytorch-object-detection-with-pre-trained-networks/)


In [1]:
# Inicialización
import torch

# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [2]:
# Definición de constantes 
import os
import numpy as np
import cv2 as cv

# esto es para que funcione matplotlib 
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

BATCH_SIZE = 64
EPOCHS = 50
IMAGE_SIZE = 256
MODELS_DIR = 'models'
CLASSES = ['__background__', 'person', 'bicycle', 'car', 'motorcycle', 
           'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
           'fire hydrant', 'N/A', 'stop sign', 'parking meter', 'bench', 
           'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 
           'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 
           'N/A', 'N/A', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 
           'snowboard', 'sports ball', 'kite', 'baseball bat', 
           'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 
           'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 
           'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 
           'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 
           'potted plant', 'bed', 'N/A', 'dining table', 'N/A', 'N/A', 
           'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 
           'cell phone', 'microwave', 'oven', 'toaster', 'sink', 
           'refrigerator', 'N/A', 'book', 'clock', 'vase', 'scissors', 
           'teddy bear', 'hair drier', 'toothbrush']

COLORS = np.random.uniform(0, 255, size=(len(CLASSES), 3))
print('Num classes:', len(CLASSES))

Num classes: 91


## Función para dibujar los *bounding boxes* en la imagen

In [5]:
# Dibujamos los objetos detectados en la imagen
import cv2 as cv

def draw_bbox(detections,image, means=None):

    for i in range(len(detections["boxes"])):

        # seleccionamos objetos con un confidence suficientemente grande
        confidence = detections["scores"][i]

        if confidence > 0.8:

            # extraemos el bounding box
            box = detections["boxes"][i].detach().cpu().numpy()
            x0, y0, x1, y1 = box.astype("int")

            # extraemos la etiqueta
            idx = int(detections["labels"][i])
            label = CLASSES[idx]

            # dibujamos el bounding box y la etiqueta en la imagen
            cv.rectangle(image, (x0, y0), (x1, y1), COLORS[idx], 2)
            y = y0 - 15 if y0 - 15 > 15 else y0 + 15
            text = f"{label} - {confidence:.2f}"
            cv.putText(image, text, (x0, y), cv.FONT_HERSHEY_SIMPLEX, 0.5, COLORS[idx], 2)
            
            # mostramos el número de objetos
            if means != None:
                text = f"Objetos: {len(means)}"
                cv.putText(image, text, (20,20), cv.FONT_HERSHEY_SIMPLEX, 1,(255,255,255), 2)
            
    return image


## Detección de objetos en imágenes con Faster R-CNN

In [4]:
# Detección de objetos en la imagen con Faster R-CNN

import numpy as np
import torch
import cv2
from torchvision.models import detection

model = detection.fasterrcnn_resnet50_fpn(pretrained=True).to(device)
model.eval()

# load the image from disk
image_name = 'images/prueba3.jpg'
image = cv.imread(image_name)

# convertimos a RGB y ponemos en formato pytorch
orig = image.copy()
image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
image = image.transpose((2, 0, 1))

# añadimos dimensión para el batch y normalizamos
image = np.expand_dims(image, axis=0)
image = image / 255.0
image = torch.FloatTensor(image)

# ejecutamos el modelo con la imagen
image = image.to(device)
detections = model(image)[0]

# dibujamos las cajas con las etiquetas y scores
output = draw_bbox(detections, orig)

# mostramos la salida
cv.imshow('Output', output)
cv.waitKey(0)
cv.destroyAllWindows()


c:\Users\gerar\anaconda3\envs\aa1\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\gerar\anaconda3\envs\aa1\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [9]:
# Procesar imágenes de un vídeo y guardar a disco
import cv2 as cv
import torch
import numpy as np

video = cv.VideoCapture("videos/new_york.mp4")

status, img = video.read()
size=(img.shape[1], img.shape[0])

fourcc = cv.VideoWriter_fourcc(*'DIVX')
video_out = cv.VideoWriter('video1.mp4', fourcc, 30.0, size)

while status:
    orig = img.copy()
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    img = img.transpose((2, 0, 1))
    
    image = np.expand_dims(img, axis=0)
    image = image / 255.0
    image = torch.FloatTensor(image).to(device)

    detections = model(image)[0]
    output = draw_bbox(detections, orig)
    
    cv.imshow('Salida', output)
    video_out.write(output)
    
    status, img = video.read()
    if cv.waitKey(1)==27:
        status = False

    
video.release()
video_out.release()
cv.destroyAllWindows()


# Seguimiento de Objetos

In [2]:
# Inicialización
import torch
import os
import numpy as np
import cv2 as cv

# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# esto es para que funcione matplotlib 
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

# definición de constantes 
CLASSES = ['__background__', 'person', 'bicycle', 'car', 'motorcycle', 
           'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
           'fire hydrant', 'N/A', 'stop sign', 'parking meter', 'bench', 
           'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 
           'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 
           'N/A', 'N/A', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 
           'snowboard', 'sports ball', 'kite', 'baseball bat', 
           'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 
           'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 
           'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 
           'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 
           'potted plant', 'bed', 'N/A', 'dining table', 'N/A', 'N/A', 
           'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 
           'cell phone', 'microwave', 'oven', 'toaster', 'sink', 
           'refrigerator', 'N/A', 'book', 'clock', 'vase', 'scissors', 
           'teddy bear', 'hair drier', 'toothbrush']

COLORS = np.random.uniform(0, 255, size=(len(CLASSES), 3))
print('Num classes:', len(CLASSES))

Using cpu device
Num classes: 91


In [6]:
# calculamos la media
def compute_means(detections):
    new_means = []
    new_labels = []
    new_bbox = []
    for i in range(len(detections["boxes"])):
        confidence = detections["scores"][i]
        if confidence > 0.8:
            box = detections["boxes"][i].detach().cpu().numpy()
            x0, y0, x1, y1 = box.astype("int")
            new_means.append(((x0+x1)/2,(y0+y1)/2))
            new_bbox.append([x0,y0,x1,y1])
            
            # extraemos la etiqueta
            label = int(detections["labels"][i])
            new_labels.append(label)

    return new_means, new_labels, new_bbox

def distance(a,b):
    d1 = a[0]-b[0]
    d2 = a[1]-b[1]
    return d1*d1+d2*d2

def match_objects(means, labels, bbox, new_means, new_labels, new_bbox):
    points = []
    point_labels = []
    for i in range(len(new_means)):
        mean = new_means[i]
        label = new_labels[i]
        box = new_bbox[i]
        
        j=0
        min_dist=9999
        min_pos=9999
        threshold = 200
        while j<len(labels):
            d = distance(mean, means[j][-1])
            if label == labels[j] and d<min_dist:
                min_dist = d
                min_pos = j
            j+=1
        
        if min_dist>threshold:
            means.append([mean])
            labels.append(label)
            bbox.append([box])
        else:
            means[min_pos].append(mean)
            bbox[min_pos].append(box)
            points.append(mean)
            point_labels.append(label)
            
    return points, point_labels
    
def draw_points(tracking, points, point_labels):
    for i in range(len(points)):
        color = (255,0,0) #COLORS[point_labels[i]] #(255,0,0)
        tracking[int(points[i][1])-1, int(points[i][0])-1]=color
        tracking[int(points[i][1])-1, int(points[i][0])]=color
        tracking[int(points[i][1]), int(points[i][0])-1]=color
        tracking[int(points[i][1]), int(points[i][0])]=color
        tracking[int(points[i][1])+1, int(points[i][0])]=color
        tracking[int(points[i][1]), int(points[i][0])+1]=color
        tracking[int(points[i][1])+1, int(points[i][0])+1]=color
        tracking[int(points[i][1])-1, int(points[i][0])+1]=color
        tracking[int(points[i][1])+1, int(points[i][0])-1]=color
        
    

In [7]:
# Procesar imágenes de un vídeo o de la webcam
import numpy as np
import torch
import cv2
from torchvision.models import detection

model = detection.fasterrcnn_mobilenet_v3_large_fpn(pretrained=True).to(device)
model.eval()

#video = cv.VideoCapture(0)
"""video = cv.VideoCapture("videos/airport.mp4")
video = cv.VideoCapture("videos/road.mp4")
video = cv.VideoCapture("videos/cars.mp4")"""
video = cv.VideoCapture("videos/people2.mp4")


status, img = video.read()
size=(img.shape[1], img.shape[0])

fourcc = cv.VideoWriter_fourcc(*'DIVX')
video_out = cv.VideoWriter('video1.mp4', fourcc, 20.0, size)

tracking = 255*np.ones((img.shape[0], img.shape[1], 3))

means = []
labels = []
bbox = []
pos = 10
while status:
    
    # adaptamos la nueva imagen
    orig = img.copy()
    original = img.copy()
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    img = img.transpose((2, 0, 1))
    
    # calculamos las predicciones
    image = np.expand_dims(img, axis=0)
    image = image / 255.0
    image = torch.FloatTensor(image).to(device)
    detections = model(image)[0]
    
    # calculamos la media
    new_means, new_labels, new_bbox = compute_means(detections)
    
    # buscamos la correspondencia anterior
    points, point_labels = match_objects(means, labels, bbox, new_means, new_labels, new_bbox)
    
    # dibujamos los nuevos puntos de las trayectorias
    draw_points(tracking, points, point_labels)
    
    # dibujamos los objetos detectados
    img_detections = draw_bbox(detections, orig, labels)
    
    # mezclamos las dos imágenes
    output = img_detections&tracking.astype(np.uint8)
    output[output>255]=255
    output[output<0]=0
    
    # mostramos el contenido de un bounding box aparte
    if len(bbox)>pos:
        x0=bbox[pos][-1][0]
        y0=bbox[pos][-1][1]
        x1=bbox[pos][-1][2]
        y1=bbox[pos][-1][3]

        if x0<x1 and y0<y1:
            subimage=original[y0:y1,x0:x1,:]
            dim = (3*subimage.shape[1], 3*subimage.shape[0])
            subimage=cv.resize(subimage,dim, interpolation = cv.INTER_AREA)
            cv.imshow('Bbox', subimage)
            
    cv.imshow('Output', output.astype(np.uint8))
    cv.imshow('Tracking', tracking)
    
    video_out.write(output)
    
    status, img = video.read()
    
    if cv.waitKey(1)==27:
        status = False
    
video.release()
video_out.release()
cv.destroyAllWindows()
